In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [53]:
day_data = pd.read_csv("day_data.csv")

In [54]:
day_data

,date,holiday,moon,arrest,num_crimes,wd
0,2010-01-01,New Year's Day,0.0,0.216216,74,Friday
1,2010-01-02,NaN,0.0,0.431373,51,Saturday
2,2010-01-03,NaN,0.0,0.340909,44,Sunday
3,2010-01-04,NaN,0.0,0.403846,52,Monday
4,2010-01-05,NaN,0.0,0.557692,52,Tuesday
...,...,...,...,...,...,...
5104,2023-12-23,NaN,0.0,0.225000,40,Saturday
5105,2023-12-24,NaN,0.0,0.071429,28,Sunday
5106,2023-12-25,Christmas Day,0.0,0.130435,23,Monday
5107,2023-12-26,NaN,0.0,0.378378,37,Tuesday


In [55]:
day_data['holiday'] = day_data['holiday'].apply(lambda x: 0 if pd.isna(x) else 1)

In [56]:
day_data['holiday']

0       1
1       0
2       0
3       0
4       0
       ..
5104    0
5105    0
5106    1
5107    0
5108    0
Name: holiday, Length: 5109, dtype: int64

In [60]:
#day_data['holiday'] = day_data['holiday'].notnull().astype(int)
day_data['date'] = pd.to_datetime(day_data['date'])
Y = day_data['arrest']
X = day_data.drop(columns=['arrest', 'date'])

X = pd.get_dummies(X, prefix='wd')


In [30]:
day_data['holiday']

0       1
1       1
2       1
3       1
4       1
       ..
5104    1
5105    1
5106    1
5107    1
5108    1
Name: holiday, Length: 5109, dtype: int64

In [61]:
X

,holiday,moon,num_crimes,wd_Friday,wd_Monday,wd_Saturday,wd_Sunday,wd_Thursday,wd_Tuesday,wd_Wednesday
0,1,0.0,74,True,False,False,False,False,False,False
1,0,0.0,51,False,False,True,False,False,False,False
2,0,0.0,44,False,False,False,True,False,False,False
3,0,0.0,52,False,True,False,False,False,False,False
4,0,0.0,52,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...
5104,0,0.0,40,False,False,True,False,False,False,False
5105,0,0.0,28,False,False,False,True,False,False,False
5106,1,0.0,23,False,True,False,False,False,False,False
5107,0,0.0,37,False,False,False,False,False,True,False


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=24)

In [63]:
lasso = LassoCV()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)
mean_squared_error(y_true=y_test, y_pred=y_pred)

0.011486100968360231

In [64]:
support_vector_regressor = SVR()
support_vector_regressor.fit(X_train, y_train)

y_pred_svr = support_vector_regressor.predict(X_test)
mean_squared_error(y_test, y_pred_svr)

0.01118453899674101